# Taller de SQLite

## 1. Introducción

Este taller práctico te guiará a través de la creación de una base de datos SQLite para gestionar una empresa, cubriendo:

- Creación de tablas y relaciones (DDL)

- Inserción y manipulación de datos (DML)

- Consultas avanzadas (JOINS, GROUP BY, HAVING, subconsultas)

- Control de acceso y permisos (DCL)

- Visualización del esquema de la base de datos
---

## 2. Creación de la Base de Datos y Tablas (DDL)

In [1]:
import sqlite3

conn = sqlite3.connect("empresa.db")

In [2]:
cursor = conn.cursor()

In [6]:
cursor.execute("""
               CREATE TABLE IF NOT EXISTS departamentos (
                   id INTEGER PRIMARY KEY AUTOINCREMENT,
                   nombre TEXT UNIQUE NOT NULL
               )
               """)

cursor.execute("""
               CREATE TABLE IF NOT EXISTS empleados (
                   id INTEGER PRIMARY KEY AUTOINCREMENT,
                   nombre TEXT NOT NULL,
                   edad INTEGER,
                   genero TEXT,
                   departamento_id INTEGER,
                   FOREIGN KEY (departamento_id) REFERENCES departamentos(id)
               )
               """)

cursor.execute("""
               CREATE TABLE IF NOT EXISTS clientes (
                   id INTEGER PRIMARY KEY AUTOINCREMENT,
                   nombre TEXT NOT NULL,
                   email TEXT UNIQUE NOT NULL,
                   telefono INTEGER
               )
               """)


cursor.execute("""
               CREATE TABLE IF NOT EXISTS proyectos (
                   id INTEGER PRIMARY KEY AUTOINCREMENT,
                   nombre TEXT NOT NULL,
                   cliente_id INTEGER,
                   FOREIGN KEY (cliente_id) REFERENCES clientes(id)
               )
               """)

cursor.execute("""
               CREATE TABLE IF NOT EXISTS facturas (
                   id INTEGER PRIMARY KEY AUTOINCREMENT,
                   proyecto_id INTEGER,
                   fecha TEXT NOT NULL,
                   monto REAL NOT NULL,
                   FOREIGN KEY (proyecto_id) REFERENCES proyectos(id)
               )
               """)

conn.commit()
conn.close()

## 3. Inserción de Datos (DML)

In [7]:
conn = sqlite3.connect("empresa.db")
cursor = conn.cursor()

cursor.execute("INSERT INTO empleados (nombre, edad, genero, departamento_id) VALUES ('Sebastian Moncada', 24, 'Male', 1)")
cursor.execute("INSERT INTO clientes (nombre, email, telefono) VALUES ('Alpina', 'alpina@email.com', 155564)")
cursor.execute("INSERT INTO proyectos (nombre, cliente_id) VALUES ('ETL', 3)")
cursor.execute("INSERT INTO facturas (proyecto_id, monto, fecha) VALUES (3, 500.25, '2025-04-08')")

conn.commit()
conn.close()

In [8]:
conn = sqlite3.connect("empresa.db")
cursor = conn.cursor()

# departamentos
cursor.executemany("INSERT INTO departamentos (nombre) VALUES (?)", [
    ('Ventas',), ('TI',), ('Logistica',), ('Recursos Humanos',)
])

# empleados
cursor.executemany("INSERT INTO empleados (nombre, edad, genero, departamento_id) VALUES (?, ?, ?, ?)", [
    ('Ana Perez', 30, 'Female', 1), ('Carlos Diaz', 45, 'Male', 2), ('Laura Perez', 32, 'Female', 2), ('Juan Sanchez', 52, 'Male', 1)
])

# clientes
cursor.executemany("INSERT INTO clientes (nombre, email, telefono) VALUES (?, ?, ?)", [
    ('Bavaria', 'bavaria@email.com', 5656541), ('Ave Maria', 'ave@mail.com', 456864541)
])


# proyectos
cursor.executemany("INSERT INTO proyectos (nombre, cliente_id) VALUES (?, ?)", [
    ('Data Science', 2), ('Mobile apps', 3)
])

# facturas
cursor.executemany("INSERT INTO facturas (proyecto_id, monto, fecha) VALUES (?, ?, ?)", [
    (2, 1500.00, '2025-01-15'), (3, 3564.25, '2024-03-22')
])

conn.commit()
conn.close()

## 4. Consultas Avanzadas

In [9]:
conn = sqlite3.connect("empresa.db")
cursor = conn.cursor()

In [14]:
cursor.execute("""
SELECT * FROM clientes
               """)
print(cursor.fetchall())

[(1, 'Alpina', 'alpina@email.com', 155564), (2, 'Bavaria', 'bavaria@email.com', 5656541), (3, 'Ave Maria', 'ave@mail.com', 456864541)]


In [15]:
cursor.execute("""
SELECT empleados.nombre, empleados.edad, departamentos.nombre AS departamento 
FROM empleados
JOIN departamentos ON empleados.departamento_id = departamentos.id
               """)
print(cursor.fetchall())

[('Sebastian Moncada', 24, 'Ventas'), ('Ana Perez', 30, 'Ventas'), ('Carlos Diaz', 45, 'TI'), ('Laura Perez', 32, 'TI'), ('Juan Sanchez', 52, 'Ventas')]


In [16]:
cursor.execute("""
SELECT clientes.nombre, SUM(facturas.monto) AS total_facturado 
FROM clientes
JOIN proyectos ON clientes.id = proyectos.cliente_id
JOIN facturas ON proyectos.id = facturas.proyecto_id
GROUP BY clientes.id
               """)
print(cursor.fetchall())

[('Bavaria', 1500.0), ('Ave Maria', 4064.5)]


In [17]:
conn.close()

In [22]:
import pandas as pd
conn = sqlite3.connect("empresa.db")

df = pd.read_sql_query("SELECT clientes.nombre, SUM(facturas.monto) AS total_facturado FROM clientes JOIN proyectos ON clientes.id = proyectos.cliente_id JOIN facturas ON proyectos.id = facturas.proyecto_id GROUP BY clientes.id", conn)
conn.close()

In [23]:
df.head()

,nombre,total_facturado
0,Bavaria,1500.0
1,Ave Maria,4064.5


## 5. Visualización del Esquema

---
## 6. Actividad

- Agregar una tabla "Tareas" asociada a empleados y proyectos.

- Consultar la cantidad de empleados por departamento.

- Actualizar la edad de un empleado.

- Eliminar un cliente y analizar el efecto en los proyectos asociados.